In [1]:
from dask.distributed import Client
import subprocess as subp
import os

import dask

In [2]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

In [3]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:43150 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [4]:
from nccl_example import NCCL_Clique, unique_id
import random
from dask.distributed import wait

In [5]:
class Dask_MPI_Demo:
    
    def __init__(self, client):
        self.client = client
    
    @staticmethod
    def func_parse_host_port_(address):
        if '://' in address:
            address = address.rsplit('://', 1)[1]
        host, port = address.split(':')
        port = int(port)
        return host, port
    
    @staticmethod
    def func_init_(workerId, nWorkers, uniqueId):
        w = dask.distributed.get_worker()
        print("Hello World! from ip=%s worker=%s/%d" % \
              (w.address, w.name, nWorkers))
        a = NCCL_Clique(workerId, nWorkers)
        a.create_clique(uniqueId)

        return a
    
    @staticmethod
    def func_get_size_(world, r):
        return world.get_clique_size()
    
    @staticmethod
    def func_get_rank_(world, r):
        return world.get_rank()
    
    @staticmethod
    def func_get_device_(world, r):
        return world.get_device()

    @staticmethod
    def func_test_all_reduce_(world, r):
        return world.test_all_reduce()
    
    
    def run_func_on_workers(self, func):
        f = [c.submit(func, a, random.random()) for i, w, a in self.clique]
        wait(f)
        return [a.result() for a in f]

    def get_workers_(self):
        return list(map(lambda x: Dask_MPI_Demo.func_parse_host_port_(x), self.client.has_what().keys()))
    
    def init(self, uniqueId):
        workers = self.get_workers_()
        workers_indices = list(zip(workers, range(len(workers))))

        self.clique = [(idx, worker, self.client.submit(Dask_MPI_Demo.func_init_, 
                                           idx, 
                                           len(workers), 
                                           uniqueId,
                                           workers=[worker]))
             for worker, idx in workers_indices]
        
    def get_clique_size(self):
        return self.run_func_on_workers(Dask_MPI_Demo.func_get_size_)

    def get_rank(self):
        return self.run_func_on_workers(Dask_MPI_Demo.func_get_rank_)
    
    def test_all_reduce(self):
        return self.run_func_on_workers(Dask_MPI_Demo.func_test_all_reduce_)

In [6]:
world_id = unique_id()

In [7]:
demo = Dask_MPI_Demo(c)
demo.init(world_id)

In [8]:
demo.get_clique_size()

[2, 2]

In [9]:
demo.get_rank()

[0, 1]

In [10]:
demo.test_all_reduce()

[True, True]